<img src="./img/header.png">
# Globus SDK
https://github.com/globusonline/globus-sdk-python


# Globus SDK Docs
http://globusonline.github.io/globus-sdk-python/


# Globus CLI
https://github.com/globusonline/globus-cli

# Requirements
- You need to be in the tutorial users group for sharing
- Installed Globus Python SDK

In [19]:
from __future__ import print_function # for python 2
from globus_sdk import TransferClient

tutorial_endpoint1 = "ddb59aef-6d04-11e5-ba46-22000b92c6ec"
tutorial_endpoint2 = "ddb59af0-6d04-11e5-ba46-22000b92c6ec"

# Configuration
First you will need to configure the client with your access credentials. You can obtain OAuth2 access tokens via the tokens.globus.org website.

In [5]:
auth_token = ''
transfer_token = ''

# Transfer Client

## Instantiate Transfer Client

In [31]:
tc = TransferClient(token=transfer_token) 

The transfer client makes REST resources avaialble via easy to use methods. 

The response from these methods wraps the HTTP response status, content type, text and JSON response body. 

In [44]:
endpoint = tc.get_endpoint(tutorial_endpoint1)
print("HTTP Status: %s" % endpoint.http_status)
print("Content Type: %s" % endpoint.content_type)
print("Data: %s" % endpoint.data) # raw response text can be obtained via endpoint.text_body or endpoint.json_body

HTTP Status: 200
Content Type: application/json
JSON: {u'DATA_TYPE': u'endpoint', u'contact_info': None, u'https_server': None, u'activated': False, u'disable_verify': False, u'host_endpoint_display_name': None, u'myproxy_server': u'myproxy.globusonline.org', u'keywords': None, u'sharing_target_root_path': None, u'DATA': [{u'is_paused': False, u'DATA_TYPE': u'server', u'hostname': u'ep1.transfer.globus.org', u'uri': u'gsiftp://ep1.transfer.globus.org:2811', u'port': 2811, u'scheme': u'gsiftp', u'is_connected': True, u'id': 207976, u'subject': None}], u'id': u'ddb59aef-6d04-11e5-ba46-22000b92c6ec', u'display_name': u'Globus Tutorial Endpoint 1', u'gcp_connected': None, u'owner_string': u'go@globusid.org', u'expires_in': -1, u'gcp_paused': None, u'subscription_id': u'964be8f5-5f9b-11e4-b64e-12313940394d', u'canonical_name': u'go#ep1', u'location': u'Automatic', u'host_endpoint_id': None, u'force_encryption': False, u'department': None, u'acl_available': False, u's3_url': None, u'public':

If an error occurs the API will throw an error.

In [43]:
try:
    endpoint = tc.get_endpoint("BAD ENDPOINT ID")
except Exception as ex:
    print("HTTP Status: %s" % ex[0])
    print("HTTP Error: %s" % ex[1])
    print("HTTP Message: %s" % ex[2])

HTTP Status: 400
HTTP Error: BadRequest
HTTP Message: Invalid endpoint name u'kyle#BAD ENDPOINT ID': Invalid characters


# Endpoint management

## Endpoint search by name

In [11]:
search_str = "Globus Tutorial Endpoint"
endpoints = tc.endpoint_search(search_str)
print("==== Displaying endpoint matches for search: '%s' ===\n"%search_str)
for r in endpoints:
    print("%s (%s)"%(r.data["display_name"], r.data["id"]))

==== Displaying endpoint matches for search: 'Globus Tutorial Endpoint' ===

BlackPearl Demo Endpoint (38a3343a-833c-11e5-9949-22000b96db58)
Globus Tutorial Endpoint 1 (ddb59aef-6d04-11e5-ba46-22000b92c6ec)
Globus Tutorial Endpoint 2 (ddb59af0-6d04-11e5-ba46-22000b92c6ec)
Globus S3 Tutorial Endpoint (cf9bcaa5-6d04-11e5-ba46-22000b92c6ec)
Globus Tutorial HTTPS Endpoint Server (30ed71b0-fc1d-11e5-a700-22000bf2d559)
None (488cae40-8c78-11e5-996f-22000b96db58)


# File operations

### Path encoding & UTF gotchas

### Autoactivate Endpoints
Here we activate the tutorial endpoints go#ep1 and go#ep2 using their endpoint_ids

In [8]:
#help(tc.endpoint_activate)
r_go1 = tc.endpoint_autoactivate(tutorial_endpoints['go#ep1'])
r_go2 = tc.endpoint_autoactivate(tutorial_endpoints['go#ep2'])

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


### Get a directory listing

In [7]:
# help(tc.operation_ls)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/'
r = tc.operation_ls(endpoint_id,path=endpoint_path)
print("==== Endpoint_ls for endpoint %s %s ===="%(endpoint_id,endpoint_path))
for item in r.data['DATA']:
    print("%s: %s [%s]"%(item['type'],item['name'],item['size']))

==== Endpoint_ls for endpoint ddb59aef-6d04-11e5-ba46-22000b92c6ec /~/shared_dir/ ====
dir: new_dir [4096]
file: Ag-microparticles.TIF [1384141]
file: ConductivityRestoration.png [4499617]
file: CuSn_battery_electrode-before.mov [3702784]


### Make directory

In [ ]:
#help(tc.operation_mkdir)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/new_dir'
r = tc.operation_mkdir(endpoint_id,path=endpoint_path)

### Rename

In [9]:
#help(tc.operation_rename)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/'
tc.operation_rename(endpoint_id, oldpath="/~/file1.txt",
                    newpath="/~/project1data.txt")

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


TransferAPIError: (404, u'NotFound', u"Path not found, Error: The source path '~/file1.txt' does not exist\n", u'Qmf4STw25')

# Task submission

### Transfer w/o sync


In [19]:
# help(tc.submit_transfer)
source_endpoint_id = tutorial_endpoints['go#ep1']
source_path = '/~/shared_dir/'

dest_endpoint_id = tutorial_endpoints['go#ep2']
dest_path = '/~/'

label = "My tutorial transfer"

transfer_items = []

## Recursively transfer source path contents
transfer_items.append(
    tc.make_submit_transfer_item(source_path,
                                 dest_path,
                                 recursive=True))
## Alternatively, transfer a specific file
# transfer_items.append(
#     tc.make_submit_transfer_item("/source/path/file.txt",
#                                  "/dest/path/file.txt"))

transfer_data = tc.make_submit_transfer_data(
    source_endpoint_id,
    dest_endpoint_id,
    transfer_items, label=label)
transfer_result = tc.submit_transfer(transfer_data)
print("task_id = ", transfer_result.data["task_id"])

task_id =  1a09c8a8-001a-11e6-a717-22000bf2d559


# Task management

### Get Task By ID

In [ ]:
r = tc.get_task(transfer_result.data['task_id'])
r.data

### Get task_list

In [36]:
# help(tc.task_list)
r = tc.task_list()
for i,item in enumerate(r):
    print(item.data)
    if r > 10:
        break

{u'files': 3, u'bytes_transferred': 9586542, u'DATA_TYPE': u'task', u'sync_level': None, u'completion_time': u'2016-04-11 19:18:02+00:00', u'canceled_by_admin_message': None, u'deadline': u'2016-04-12 19:18:00+00:00', u'destination_endpoint': u'go#ep2', u'bytes_checksummed': 0, u'delete_destination_extra': False, u'nice_status_details': None, u'source_endpoint_id': u'ddb59aef-6d04-11e5-ba46-22000b92c6ec', u'request_time': u'2016-04-11 19:18:00+00:00', u'nice_status': None, u'subtasks_expired': 0, u'subtasks_canceled': 0, u'label': u'My tutorial transfer', u'faults': 0, u'destination_endpoint_display_name': u'Globus Tutorial Endpoint 2', u'subtasks_total': 6, u'source_endpoint_display_name': u'Globus Tutorial Endpoint 1', u'command': u'API 0.10', u'nice_status_expires_in': None, u'subtask_link': {u'href': u'task/1a09c8a8-001a-11e6-a717-22000bf2d559/subtask_list?format=json', u'resource': u'subtask list', u'rel': u'child', u'DATA_TYPE': u'link', u'title': u'child subtask list'}, u'owner_

### Update task by id (label, deadline)

In [ ]:
help(tc.update_task)

### Cancel task by id

In [ ]:
help(tc.cancel_task)

### Get event list for task

### Get task pause info

# Bookmarks

# Create a Bookmark

In [ ]:
bookmark_name = "My Tutorial Bookmark"
endpoint_id = tutorial_endpoints['go#ep1']
endpoint_path = '/'
r = tc.create_bookmark({"endpoint_id":endpoint_id, "path":endpoint_path,"name":bookmark_name})
r

# Delete a Bookmark

In [ ]:
tc.delete_bookmark(r.data['id'])

### CRUD + list

# Shared endpoints

### Access_manager role

### Get list off access rules

### Get access rule by id

### Create access rule

### Update access rule

### Delete access rule